# benchmarking on omnipath


In [1]:
#! lamin init --storage ./lamin-intro --schema bionty 

import scanpy as sc
from huggingface_hub import hf_hub_download

from scprint2 import scPRINT2
from scprint.tasks import GNInfer
from scdataloader import Preprocessor

from bengrn import BenGRN
import scanpy as sc
from anndata.utils import make_index_unique

from bengrn.base import train_classifier
from bengrn import compute_genie3
from grnndata import utils as grnutils
import joblib
%load_ext autoreload
%autoreload 2 

import torch
torch.set_float32_matmul_precision('medium')

→ connected lamindb: jkobject/scprint2


In [2]:
model_checkpoint_file = hf_hub_download(
    repo_id="jkobject/scPRINT", filename=f"v2-medium.ckpt"
)
#model_checkpoint_file = "../data/temp/o2uniqsx/epoch=18-step=133000.ckpt"
# "/pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/scprint_scale/o2uniqsx/checkpoints/last.ckpt"
#'../data/temp/o2uniqsx/epoch=18-step=133000.ckpt',
#"../../xzp23r4p.ckpt"
model_checkpoint_file

'/home/ml4ig1/.cache/huggingface/hub/models--jkobject--scPRINT/snapshots/d79fad0ff77e19d9ba0c2fad9dfc993b37e92d6d/v2-medium.ckpt'

In [3]:
m = torch.load(model_checkpoint_file)
if "prenorm" in m['hyper_parameters']:
    m['hyper_parameters'].pop("prenorm")
    torch.save(m, model_checkpoint_file)
if "label_counts" in m['hyper_parameters']:
    model = scPrint.load_from_checkpoint(model_checkpoint_file, precpt_gene_emb=None, classes=m['hyper_parameters']['label_counts'])
else:
    model = scPrint.load_from_checkpoint(model_checkpoint_file, precpt_gene_emb=None)

RuntimeError caught: scPrint is not attached to a `Trainer`.


In [4]:
CELLTYPES = [
    'kidney distal convoluted tubule epithelial cell',
    'kidney loop of Henle thick ascending limb epithelial cell',
    'kidney collecting duct principal cell',
    'mesangial cell',
    'blood vessel smooth muscle cell',
    'podocyte',
    'macrophage',
    'leukocyte',
    'kidney interstitial fibroblast',
    'endothelial cell'
]

NUM_GENES = 5000
MAXCELLS = 1024

In [5]:
#only dataset dropped
adata = sc.read("../data/temp/yBCKp6HmXuHa0cZptMo7.h5ad", backup_url="https://datasets.cellxgene.cziscience.com/02f8bb42-0322-4906-b9ad-b92a5ce73a2d.h5ad")

adata.var["isTF"] = False
adata.var.loc[adata.var.symbol.isin(grnutils.TF), "isTF"] = True
adata

AnnData object with n_obs × n_vars = 15728 × 70116
    obs: 'donor_id', 'self_reported_ethnicity_ontology_term_id', 'organism_ontology_term_id', 'sample_uuid', 'sample_preservation_method', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'tissue_section_uuid', 'tissue_section_thickness', 'library_uuid', 'assay_ontology_term_id', 'mapped_reference_annotation', 'is_primary_data', 'cell_type_ontology_term_id', 'author_cell_type', 'disease_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'cell_culture', 'nnz', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'leiden_3', 'leide

In [6]:
adata.obs['organism_ontology_term_id'] = "NCBITaxon:9606"
preprocessor = Preprocessor(force_preprocess=True, skip_validate=True, do_postp=False)
adata = preprocessor(adata.copy())

Dropping layers:  KeysView(Layers with keys: clean)
checking raw counts
removed 0 non primary cells, 15728 renamining
filtered out 5865 cells, 9863 renamining
Removed 0 genes.
startin QC
Seeing 1531 outliers (15.52% of total dataset):
done
AnnData object with n_obs × n_vars = 9863 × 70704
    obs: 'donor_id', 'self_reported_ethnicity_ontology_term_id', 'organism_ontology_term_id', 'sample_uuid', 'sample_preservation_method', 'tissue_ontology_term_id', 'development_stage_ontology_term_id', 'tissue_section_uuid', 'tissue_section_thickness', 'library_uuid', 'assay_ontology_term_id', 'mapped_reference_annotation', 'is_primary_data', 'cell_type_ontology_term_id', 'author_cell_type', 'disease_ontology_term_id', 'sex_ontology_term_id', 'suspension_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage', 'cell_culture', 'nnz', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_g

In [7]:
# just in case when restarting after a crash and on the same notebook
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [8]:
metrics = {}
clf_omni = None
shapes = []
for celltype in CELLTYPES:
    # RANDOM EXPR means that we will randomly choose different genes for each cell
    # we then end up averaging across and getting a genome-wide gene network
    # grn_inferer = GNInfer(how="random expr",
    #                     preprocess="softmax",
    #                     head_agg='mean',
    #                     filtration="none",
    #                     forward_mode="none",
    #                     num_genes=2200,
    #                     max_cells=MAXCELLS,
    #                     doplot=False,
    #                     batch_size=32,
    #                     layer=list(range(model.nlayers))[:],
    #                     )
    # grn = grn_inferer(model, adata[adata.X.sum(1) > 500], cell_type=celltype)
    # grn.var.index = make_index_unique(grn.var['symbol'].astype(str))
    # metrics[celltype+'_scprint_full'] = BenGRN(grn).scprint_benchmark()
    # del grn
    grn_inferer = GNInfer(how="most var across",
                        layer=list(range(model.nlayers))[:],
                        preprocess="softmax",
                        head_agg='none',
                        filtration="none",
                        forward_mode="none",
                        num_genes=NUM_GENES,
                        max_cells=1024,
                        drop_unexpressed=True,
                        doplot=False,
                        batch_size=16,
                        )
    grn = grn_inferer(model, adata[adata.X.sum(1) > 500], cell_type=celltype)
    grn.var.index = make_index_unique(grn.var['symbol'].astype(str))
    grn.varp['all'] = grn.varp['GRN']
    grn.varp['GRN'] = grn.varp['GRN'][:,:,:].mean(-1)
    metrics[celltype+'_scprint_mean'] = BenGRN(grn).scprint_benchmark()
    grn.varp['GRN'][~grn.var.isTF,:]=0
    metrics[celltype+'_scprint_tf_mean'] = BenGRN(grn).scprint_benchmark()
    if clf_omni == None:
        grn.varp['GRN'] = grn.varp['all']
        _, m, clf_omni = train_classifier(grn, C=1, train_size=0.6, class_weight={
                                        1: 200, 0: 1}, shuffle=True, max_iter=200, return_full=False)
        joblib.dump(clf_omni, 'clf_omni.pkl')
    grn.varp['GRN'] = grn.varp['all'][:,:,clf_omni.coef_[0] > 0].mean(-1)
    metrics[celltype+'_scprint_class'] = BenGRN(grn).scprint_benchmark()
    grn.varp['GRN'][~grn.var.isTF,:]=0
    metrics[celltype+'_scprint_tf_class'] = BenGRN(grn).scprint_benchmark()
    shapes.append(grn.shape[1])
    del grn

> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/grn.py(151)__call__()
    149         pdb.set_trace()
    150 
--> 151         subadata = self.predict(model, adata, self.layer, cell_type)
    152         adjacencies = self.aggregate(model.attn.get(), model.genes)
    153         if self.head_agg == "none":



... storing 'organism_ontology_term_id' as categorical
... storing 'symbol_x' as categorical
... storing 'symbol_y' as categorical
... storing 'ncbi_gene_ids' as categorical
... storing 'biotype_y' as categorical
... storing 'synonyms_y' as categorical
... storing 'description_y' as categorical
... storing 'organism' as categorical
... storing 'symbol' as categorical
... storing 'biotype' as categorical
... storing 'synonyms' as categorical
... storing 'description' as categorical


/home/ml4ig1/Documents code/scDataLoader/scdataloader/utils.py:400: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  organismdf = pd.concat(organismdf)
100%|██████████| 54/54 [00:12<00:00,  4.38it/s]


> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/grn.py(152)__call__()
    150 
    151         subadata = self.predict(model, adata, self.layer, cell_type)
--> 152         adjacencies = self.aggregate(model.attn.get(), model.genes)
    153         if self.head_agg == "none":
    154             return self.save(

> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/grn.py(153)__call__()
    151         subadata = self.predict(model, adata, self.layer, cell_type)
    152         adjacencies = self.aggregate(model.attn.get(), model.genes)
--> 153         if self.head_agg == "none":
    154             return self.save(
    155                 adjacencies[self.n_cell_embs :, self.n_cell_embs :, :],

(5009, 5009, 32)
array([[[5.23121071e-06, 6.67405914e-07, 2.70848413e-05, ...,
         4.57341898e-08, 2.63622284e-01, 1.70826630e-15],
        [7.17682553e-08, 8.41931183e-07, 1.03032187e-04, ...,
         1.37236857e-21, 9.65094776e-04, 3.35029668e-20],
        [1.53338465e-06, 6.8507040

NameError: name 'make_index_unique' is not defined

In [7]:
metrics

{'kidney distal convoluted tubule epithelial cell_scprint_full': {'TF_enr': True,
  'enriched_terms_Targets': ['celltype.gmt__Effector CD8+ T cells',
   'celltype.gmt__Naive CD8+ T cells',
   'celltype.gmt__Memory CD8+ T cells',
   'celltype.gmt__-T cells',
   'celltype.gmt__Myeloid Dendritic cells'],
  'significant_enriched_TFtargets': 62.5,
  'precision': 0.0008896322223674437,
  'recall': 1.0,
  'rand_precision': 0.0008897036558203398,
  'auprc': 0.0010112919661082434,
  'ap': 0.001011442546500521,
  'epr': 1.794288323583073},
 'kidney distal convoluted tubule epithelial cell_scprint_mean': {'enriched_terms_Regulators': ['celltype.gmt__Distal tubule cells',
   'celltype.gmt__Beta cells',
   'celltype.gmt__Cholangiocytes',
   'celltype.gmt__Loop of Henle cells',
   'celltype.gmt__Proximal tubule cells',
   'celltype.gmt__Ductal cells',
   'celltype.gmt__Gamma (PP) cells'],
  'TF_enr': True,
  'enriched_terms_Targets': ['celltype.gmt__Acinar cells',
   'celltype.gmt__Distal tubule cel

In [8]:
shapes

[5000, 5000, 5000, 3061, 5000, 5000, 5000, 3510, 4460, 2994]

In [9]:
False
True
True
False
True
False
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
True
False
False
False
True
True
True
True
False
True
False
False
False
False
False
False
False
False
False
True
False

True

In [11]:
import pandas as pd

In [12]:
res = []
for k, v in metrics.items():
    res.append([k.split('_')[-1], k.split('_')[0], v['epr'], v['auprc'], v['rand_precision'], v['significant_enriched_TFtargets'], v.get('TF_enr', False), 'tf_' in k])

df = pd.DataFrame(res, columns=['tools','name','EPR', 'AUPRC', 'RAND', 'TF_targ', 'TF_enr', 'TF_only'])
df

,tools,name,EPR,AUPRC,RAND,TF_targ,TF_enr,TF_only
0,full,kidney distal convoluted tubule epithelial cell,1.794288,0.001011,0.000890,62.500000,True,False
1,mean,kidney distal convoluted tubule epithelial cell,2.662047,0.000876,0.000826,11.764706,True,False
2,mean,kidney distal convoluted tubule epithelial cell,4.246105,0.001663,0.000826,17.647059,True,True
3,class,kidney distal convoluted tubule epithelial cell,4.979990,0.000994,0.000826,2.941176,True,False
4,class,kidney distal convoluted tubule epithelial cell,12.416586,0.002227,0.000826,8.823529,True,True
5,full,kidney loop of Henle thick ascending limb epit...,1.728841,0.001023,0.000890,67.857143,True,False
6,mean,kidney loop of Henle thick ascending limb epit...,1.721485,0.000883,0.000834,11.538462,True,False
7,mean,kidney loop of Henle thick ascending limb epit...,10.351980,0.002635,0.000834,7.692308,True,True
8,class,kidney loop of Henle thick ascending limb epit...,5.319020,0.000959,0.000834,7.692308,False,False
9,class,kidney loop of Henle thick ascending limb epit...,18.579117,0.003498,0.000834,3.846154,True,True
